In [94]:
import numpy as np
import math as m
from itertools import combinations


def factors(n): #розклад числа на прості множники
    primfact = []
    if n < 0 : 
        primfact.append(-1)
        n = -1 * n
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfact.append(d)  
            n //= d
        d += 1
    if n > 1:
        primfact.append(n)
    return primfact

In [119]:
def update_vec(b_i, Base): #перетворюємо b на вектор v
    fact = factors(b_i)
    v = [0 for i in range(len(Base))]
    for i in range(len(Base)): 
        if Base[i] in fact:
            v[i] = fact.count(Base[i])
    return v


def num_to_vec(b_lst, Base):
    V = []
    fact = factors(b_lst[-1]) #розклад b^2 на прості множники
    if (len(fact) == 1) or (len(fact) == 2 and fact[0] == -1):
        for b_i in b_lst[1:]:
            v_i = update_vec(b_i, Base)
            V.append(v_i)
        V = np.array([np.array(vi) for vi in V])
        return V[:-1], Base
    else:
        if not all(el in Base for el in fact): #перевіряємо чи є ці множники у факторній базі
            Base = list(np.unique(fact + Base)) #додаємо множники в факторну базу 
        for b_i in b_lst[1:]:
            v_i = update_vec(b_i, Base)
            V.append(v_i)
        V = np.array([np.array(vi) for vi in V])
        return V, Base

def better(b,n):
    if b < n/2 :
        return b
    else: 
        return (n-b)*(-1)

def display(lst):
    for i in lst:
        print(i)

In [133]:
def find_v(Vec):
    n = len(Vec)
    x = range(n)
    res = []
    for i in range(2, n+1):
        comb = list(combinations(x, i))
        for c in comb:
            s = [0 for i in range(len(Vec[0]))]
            for j in c:
                if np.any(Vec[j]): #перевірка щоб не враховувати v=[00..0]
                    s = (s + Vec[j]) % 2
            if np.sum(s) == 0:
                res.append(list(c))
    return res    

In [156]:
def brillhart_morrison(n, u, v):
    Base = []
    Vect = []
    a = []
    al = 0
    b = [1]
    b_2 = [1]
    D = n
    sq_D = m.sqrt(n)
    i = 0
    solved = False
    while not solved:
        if i == 0:
            al = m.sqrt(n)
            a.append(int(al))
            u = u + a[-1]*v
            b.append((a[-1] * b[-1])%n)
            d = (b[-1]**2)%n
            b_2.append(better(d, n))
            Vect, Base = num_to_vec(b_2, Base) #оновлюємо факт.базу і векторні представлення
#             print("\ni=",i)
#             print("al=", al)
#             print("a=",a)
#             print("b=",b)
#             print("b_2=", b_2)
#             print("Vect=", Vect)
#             print("Base=", Base)
            i += 1
        else: 
            v = (D - u**2) / v
            al = (sq_D + u) / v
            a.append(int(al))
            u = a[-1] * v - u
            b.append((a[-1] * b[-1] + b[-2])%n)
            d = (b[-1]**2)%n
            b_2.append(better(d, n))
            Vect, Base = num_to_vec(b_2, Base) #оновлюємо факт.базу і векторні представлення
#             print("\ni=",i)
#             print("a=",a)
#             print("b=",b)
#             print("b_2=", b_2)
#             print("Vect=", Vect)
#             print("Base=", Base)
            nums = find_v(Vect)
            if len(nums) > 0:
                for el in nums:
                    X = 1
                    for j in el:
                        X = (X * b[j+1]) % n
                    k = 1
                    for j in el:
                        k = (k * b_2[j+1]) % n
                    Y = int(m.sqrt(k))

                    p = m.gcd((X + Y) % n, n)
                    q = m.gcd((X - Y) % n, n)

                    if (p > 1 and p < n) and (q > 1 and q < n):
                        solved = True
                        return p, q, el
            i += 1
    
# n = 25511
n = 2956217
u = 0
v = 1
p, q, el = brillhart_morrison(n, u, v)
print("\nANSWER:","\np=", p,"\nq=", q, "\nvectors indexes=", el)


ANSWER: 
p= 268747 
q= 11 
vectors indexes= [4, 9, 10, 15]
